In [83]:
%pip install tensorflow==2.7.0 tensorflow-gpu==2.7.0 opencv-python mediapipe scikit-learn sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\dangl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [84]:
import numpy as np
import cv2
import os
import mediapipe as mp

In [85]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [86]:
def mediapipe_detection(input_image, model):
    # Convert image from BGR to RGB
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    # Make prediction
    detection_results = model.process(input_image)
    # Convert image back to BGR
    input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)

    return input_image, detection_results

In [87]:
def draw_landmarks(input_image, detection_results):
    # Draw face connections
    mp_drawing.draw_landmarks(input_image, detection_results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)

In [88]:
def draw_styled_landmarks(input_image, detection_results):
    # Set drawing specifications for face connections
    face_points = mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=1) 
    face_lines = mp_drawing.DrawingSpec(color=(255,255,255), thickness=1)

    # Draw face connections
    mp_drawing.draw_landmarks(input_image, detection_results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, face_points, face_lines)


In [89]:
# Extract keypoints into flattened arrays
def extract_keypoints (detection_results):

    face = np.array([[results.x, results.y, results.z]
    for results in detection_results.face_landmarks.landmark]).flatten() if detection_results.face_landmarks else np.zeros(468*3)

    return np.concatenate([face])

In [90]:
# Path to store exported np array
DATA_PATH = os.path.join('EmotionData')

# Gesture that will be detected 
emotions = np.array(['Happy','Sad','Angry','Neutral','Surprised'])

# Number of sequences of data
n_frames = 20
# number of frames in each sequence 
length_frames = 3

In [91]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [92]:
label_map = {label:num for num, label in enumerate(emotions)}

In [93]:
sequences, labels = [], []
for emotion in emotions:
    for frame in np.array(os.listdir(os.path.join(DATA_PATH, emotion))).astype(int):
        window = []
        for frame_num in range(length_frames):
            res = np.load(os.path.join(DATA_PATH, emotion, str(frame), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[emotion])

In [94]:
X = np.array(sequences)

In [95]:
y = to_categorical(labels).astype(int)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [98]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [99]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(3,1404)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(emotions.shape[0], activation='softmax'))

In [100]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [101]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
3/3 [==============================] - 3s 27ms/step - loss: 1.6121 - categorical_accuracy: 0.1789
Epoch 2/1000
3/3 [==============================] - 0s 22ms/step - loss: 1.6102 - categorical_accuracy: 0.1789
Epoch 3/1000
3/3 [==============================] - 0s 27ms/step - loss: 1.6105 - categorical_accuracy: 0.2105
Epoch 4/1000
3/3 [==============================] - 0s 30ms/step - loss: 1.6069 - categorical_accuracy: 0.2105
Epoch 5/1000
3/3 [==============================] - 0s 19ms/step - loss: 1.6059 - categorical_accuracy: 0.2105
Epoch 6/1000
3/3 [==============================] - 0s 22ms/step - loss: 1.6053 - categorical_accuracy: 0.2316
Epoch 7/1000
3/3 [==============================] - 0s 24ms/step - loss: 1.6141 - categorical_accuracy: 0.1789
Epoch 8/1000
3/3 [==============================] - 0s 22ms/step - loss: 1.6037 - categorical_accuracy: 0.2211
Epoch 9/1000
3/3 [==============================] - 0s 27ms/step - loss: 1.5980 - categorical_accuracy: 0.2105
E

In [102]:
sequence = []
current_emotion = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, input_frame = cap.read()

        # Make detections
        input_image, detection_results = mediapipe_detection(input_frame, holistic)
        print(detection_results)

        # Draw landmarks
        draw_styled_landmarks(input_image, detection_results)

        keypoints = extract_keypoints(detection_results)
        sequence.append(keypoints)
        sequence = sequence[-3:]

        if len(sequence) == 3:
            result = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(emotions[np.argmax(result)])
            predictions.append(np.argmax(result))

            # Only returns if new emotion detected
            if (
                np.unique(predictions[-3:])[0] == np.argmax(result)
                and result[np.argmax(result)] > threshold
            ):
                if len(current_emotion) > 0: 
                    if emotions[np.argmax(result)] != current_emotion[-1]:
                        current_emotion.append(emotions[np.argmax(result)])
                else:
                    current_emotion.append(emotions[np.argmax(result)])

            if len(current_emotion) > 1: 
                current_emotion = current_emotion[-1:]

        cv2.rectangle(input_image, (0,0), (0, 0), (0, 0, 0))
        cv2.putText(input_image, ' '.join(current_emotion), (5,35), 
        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show
        cv2.imshow('OpenCV Feed', input_image)

        # Break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Sad
<class 'mediapipe.python.solutio